In [11]:
import os
from dotenv import load_dotenv

import torch
print("Torch version:", torch.__version__)
import torch.nn as nn
from clearml import Task
from clearml import Dataset
import pandas as pd

from torch import nn

Torch version: 2.7.0+cpu


In [3]:
# Загрузка переменных окружения из файла .env
load_dotenv()

Task.set_credentials(
    api_host=os.getenv('CLEARML_API_HOST'),
    web_host=os.getenv('CLEARML_WEB_HOST'),
    files_host=os.getenv('CLEARML_FILES_HOST'),
    key=os.getenv('CLEARML_API_ACCESS_KEY'),
    secret=os.getenv('CLEARML_API_SECRET_KEY')
)

In [4]:
# Инициализируем задачу (Task) в ClearML
task = Task.init(project_name="SimpleNN Project", task_name="Experiment #1")
task

ClearML Task: created new task id=a7aa62a6b95a4d90aa9b3d741e3dca3b
ClearML results page: https://app.clear.ml/projects/2dc955db01d24082b4d83b5802578449/experiments/a7aa62a6b95a4d90aa9b3d741e3dca3b/output/log


ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring


In [5]:
# Подключаем к задаче параметры (например, гиперпараметры)
config = {"learning_rate": 0.01, "batch_size": 16, "epochs": 3}
task.connect(config) 

{'learning_rate': 0.01, 'batch_size': 16, 'epochs': 3}

# Задание 1

Выполните задание на своём устройстве. 

Добавьте в тренировочный цикл ClearML-логирование. Запустите в своём окружении код:

In [6]:
# 1. Определяем класс SimpleNN

class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=5, out_features=5)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(in_features=5, out_features=1)


    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


# 2. Инициализация ClearML задачи
task = Task.init(project_name="SimpleNN Project", task_name="Experiment #1")
config = {"learning_rate": 0.01, "batch_size": 4, "epochs": 100}
task.connect(config)


# 3. Создаём модель, оптимизатор и функцию потерь
model = SimpleNN().to(torch.device('cpu'))
optimizer = torch.optim.SGD(model.parameters(), lr=config["learning_rate"])
criterion = nn.MSELoss()


# 4. Генерируем случайные входные данные и метки
torch.manual_seed(0)
X = torch.randn(3, 5)
y = torch.randn(3, 1)


# 5. Обучение модели
for epoch in range(config["epochs"]):
    outputs = model(X)
    loss = criterion(outputs, y)


    # Шаг обучения
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    # Логируем loss с указанием итерации
    task.get_logger().report_scalar("Loss", "train", loss.item(), epoch)
    print(f"Epoch {epoch + 1}/{config['epochs']}, Epoch {epoch}, Loss: {loss.item():.4f}")

Epoch 1/100, Epoch 0, Loss: 0.1321
Epoch 2/100, Epoch 1, Loss: 0.1238
Epoch 3/100, Epoch 2, Loss: 0.1164
Epoch 4/100, Epoch 3, Loss: 0.1096
Epoch 5/100, Epoch 4, Loss: 0.1034
Epoch 6/100, Epoch 5, Loss: 0.0978
Epoch 7/100, Epoch 6, Loss: 0.0927
Epoch 8/100, Epoch 7, Loss: 0.0880
Epoch 9/100, Epoch 8, Loss: 0.0837
Epoch 10/100, Epoch 9, Loss: 0.0798
Epoch 11/100, Epoch 10, Loss: 0.0762
Epoch 12/100, Epoch 11, Loss: 0.0729
Epoch 13/100, Epoch 12, Loss: 0.0699
Epoch 14/100, Epoch 13, Loss: 0.0671
Epoch 15/100, Epoch 14, Loss: 0.0645
Epoch 16/100, Epoch 15, Loss: 0.0621
Epoch 17/100, Epoch 16, Loss: 0.0598
Epoch 18/100, Epoch 17, Loss: 0.0577
Epoch 19/100, Epoch 18, Loss: 0.0558
Epoch 20/100, Epoch 19, Loss: 0.0540
Epoch 21/100, Epoch 20, Loss: 0.0523
Epoch 22/100, Epoch 21, Loss: 0.0507
Epoch 23/100, Epoch 22, Loss: 0.0493
Epoch 24/100, Epoch 23, Loss: 0.0479
Epoch 25/100, Epoch 24, Loss: 0.0466
Epoch 26/100, Epoch 25, Loss: 0.0453
Epoch 27/100, Epoch 26, Loss: 0.0442
Epoch 28/100, Epoch 

# Задание 2. Работа с датасетами через ClearML

Добавьте датасет в ClearML удобным вам способом.

In [7]:
# Добавьте датасет в ClearML

dataset = Dataset.create(dataset_project="SimpleNN Project", dataset_name="synthetic_data")


# Генерируем случайные данные как в прошлом задании
torch.manual_seed(0)
X = torch.randn(3, 5)   # форма [3, 5]
y = torch.randn(3, 1)   # форма [3, 1]


# Превращаем X и y в pandas.DataFrame
# Сначала приводим к numpy, затем создаём DataFrame

df = pd.DataFrame(
    X.numpy(),
    columns=[f"f{i+1}" for i in range(5)]   # f1, f2, f3, f4, f5
)
df["label"] = y.numpy()  # добавляем колонку "label"


df.to_csv("./data/data.csv", index=False)
dataset.add_files("./data/data.csv")
dataset.upload()
dataset.finalize()

df

ClearML results page: https://app.clear.ml/projects/80631adacb78415a947c0f6c041322da/experiments/8f8850275dc14d57b11a2ea40fa9248a/output/log
ClearML dataset page: https://app.clear.ml/datasets/simple/80631adacb78415a947c0f6c041322da/experiments/8f8850275dc14d57b11a2ea40fa9248a
Uploading dataset changes (1 files compressed to 250 B) to https://files.clear.ml
File compression and upload completed: total size 250 B, 1 chunk(s) stored (average size 250 B)


,f1,f2,f3,f4,f5,label
0,1.540996,-0.293429,-2.178789,0.568431,-1.084522,0.122701
1,-1.398595,0.403347,0.838026,-0.719258,-0.403344,-0.566317
2,-0.596635,0.182036,-0.856675,1.100604,-1.071187,0.373115


In [ ]:
# Чтобы в дальнейшем использовать датасет в коде обучения

dataset = Dataset.get(dataset_project="SimpleNN Project", dataset_name="synthetic_data")
data_path = dataset.get_local_copy()
print(data_path)

# Локальный файл сидит тут:
# C:/Users/Ushankova.M.Ju/.clearml/cache/storage_manager/datasets/ds_8f8850275dc14d57b11a2ea40fa9248a

C:/Users/Ushankova.M.Ju/.clearml/cache/storage_manager/datasets/ds_8f8850275dc14d57b11a2ea40fa9248a


In [9]:
# Проверяем содержимое директории
files = os.listdir(data_path)
print("Содержимое директории:", files)

Содержимое директории: ['data.csv']


In [ ]:
# Теперь считываем data.csv из папки data_path
df = pd.read_csv(f'{data_path}/{files[0]}')
# df = pd.read_csv(data_path + '/data.csv')
df

,f1,f2,f3,f4,f5,label
0,1.540996,-0.293429,-2.178789,0.568431,-1.084522,0.122701
1,-1.398596,0.403347,0.838026,-0.719258,-0.403344,-0.566318
2,-0.596635,0.182036,-0.856675,1.100604,-1.071187,0.373115


# Задание 3

Выполните задание локально на своём устройстве: скопируйте код ниже, реализуйте шаги и проанализируйте результаты. 

Проверить задание вы сможете самостоятельно. В конце выполнения нажмите кнопку «Готово» и увидите авторское решение — с ним вы можете сверить свой вариант. 

Что нужно сделать

1. Инициализируйте ClearML-задание с именем "Experiment #1" и project_name "SimpleNN Project", чтобы залогировать метрику. Используйте Task из clearml для инициализации и получения логгера.
1. Сделайте логи для эксперимента в 20 эпох, с lr, равным 0.01, и batch_size, равным 2. В качестве данных для обучения используйте сохранённый датасет из предыдущего задания этого урока.

In [ ]:
# 1. Определяем класс SimpleNN
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=5, out_features=5)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(in_features=5, out_features=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 2. Инициализация ClearML задачи
task = Task.init(project_name="SimpleNN Project", task_name="Experiment #1")
config = {
    "learning_rate": 0.01, 
    "batch_size": 2, 
    "epochs": 20
    }

# Подключение к task
task.connect(config)

# 3. Создаём модель, оптимизатор и функцию потерь
model = SimpleNN().to(torch.device('cpu'))
optimizer = torch.optim.SGD(model.parameters(), lr=config["learning_rate"])
criterion = nn.MSELoss()

# 4. Получаем датасет
dataset = Dataset.get(dataset_project="SimpleNN Project", dataset_name="synthetic_data")
data_path = dataset.get_local_copy()
print("Путь к данным:", data_path)

# Загрузка данных из CSV
df = pd.read_csv(data_path + '/data.csv')  # Убедитесь, что файл называется 'data.csv'

# Генерация данных (или используйте данные из df)
torch.manual_seed(0)
X = torch.tensor(df.iloc[:, :-1].values, dtype=torch.float32)  # Все столбцы, кроме последнего
y = torch.tensor(df.iloc[:, -1].values.reshape(-1, 1), dtype=torch.float32)  # Последний столбец

# 5. Обучение модели
for epoch in range(config["epochs"]):
    outputs = model(X)
    loss = criterion(outputs, y)

    # Шаг обучения
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # print(loss.item())
    # Логируем loss через task с указанием итерации
    task.get_logger().report_scalar("Loss", "train", iteration=epoch, value=loss.item())
    
    print(f"Epoch {epoch + 1}/{config['epochs']}, Loss: {loss.item():.4f}")

# Завершение задачи
task.close()

Путь к данным: C:/Users/Ushankova.M.Ju/.clearml/cache/storage_manager/datasets/ds_8f8850275dc14d57b11a2ea40fa9248a
0.6173222661018372
Epoch 1/20, Loss: 0.6173
0.5845122337341309
Epoch 2/20, Loss: 0.5845
0.5548596382141113
Epoch 3/20, Loss: 0.5549
0.5279551148414612
Epoch 4/20, Loss: 0.5280
0.5034551024436951
Epoch 5/20, Loss: 0.5035
0.48106876015663147
Epoch 6/20, Loss: 0.4811
0.46054887771606445
Epoch 7/20, Loss: 0.4605
0.44168367981910706
Epoch 8/20, Loss: 0.4417
0.42429134249687195
Epoch 9/20, Loss: 0.4243
0.4082144796848297
Epoch 10/20, Loss: 0.4082
0.3933165967464447
Epoch 11/20, Loss: 0.3933
0.37947845458984375
Epoch 12/20, Loss: 0.3795
0.3665960729122162
Epoch 13/20, Loss: 0.3666
0.35457754135131836
Epoch 14/20, Loss: 0.3546
0.34334221482276917
Epoch 15/20, Loss: 0.3433
0.3328186273574829
Epoch 16/20, Loss: 0.3328
0.3229433000087738
Epoch 17/20, Loss: 0.3229
0.31365978717803955
Epoch 18/20, Loss: 0.3137
0.3049176037311554
Epoch 19/20, Loss: 0.3049
0.29667162895202637
Epoch 20/20